# Q2 - Constructor Pit Stops - Data Validation and Sanity Checks

In [6]:
import pandas as pd

# read csv file
df_pitstops = pd.read_csv('/Users/frankdong/Documents/Analytics Local/williams-racing-strategies/processed_data/constructor-pit-stops.csv')

# dataframe basic info
print(df_pitstops.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1286 entries, 0 to 1285
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   race_id          1286 non-null   int64  
 1   gp_year          1286 non-null   int64  
 2   gp_name          1286 non-null   object 
 3   gp_round         1286 non-null   int64  
 4   driver_id        1286 non-null   int64  
 5   driver_name      1286 non-null   object 
 6   constructor      1286 non-null   object 
 7   constructor_ref  1286 non-null   object 
 8   is_williams      1286 non-null   bool   
 9   stop_number      1286 non-null   int64  
 10  lap_number       1286 non-null   int64  
 11  time_of_stop     1286 non-null   object 
 12  pit_duration     1286 non-null   object 
 13  pit_duration_ms  1286 non-null   int64  
 14  pit_duration_s   1286 non-null   float64
dtypes: bool(1), float64(1), int64(7), object(6)
memory usage: 142.0+ KB
None


## Summary of processed dataset 'grid-to-finish.csv'

- Filepath: /Users/frankdong/Documents/Analytics Local/williams-racing-strategies/processed_data/constructor-pit-stops.csv *(potentially fix from absolute to relative path later?)*
- Range: 1286 entries, 0 to 1285.
- Columns: 15
- Data types: float64(1), int64(7), object(6), bool(1) *(objects are strings)*
- Memory usage: 142.0+ KB

## Column data types

In [7]:
print(df_pitstops.dtypes)

race_id              int64
gp_year              int64
gp_name             object
gp_round             int64
driver_id            int64
driver_name         object
constructor         object
constructor_ref     object
is_williams           bool
stop_number          int64
lap_number           int64
time_of_stop        object
pit_duration        object
pit_duration_ms      int64
pit_duration_s     float64
dtype: object


## Missing or null values

In [8]:
df_pitstops.isnull().sum() # No nulls present across the dataset!

race_id            0
gp_year            0
gp_name            0
gp_round           0
driver_id          0
driver_name        0
constructor        0
constructor_ref    0
is_williams        0
stop_number        0
lap_number         0
time_of_stop       0
pit_duration       0
pit_duration_ms    0
pit_duration_s     0
dtype: int64

## Check for duplicates

In [9]:
df_pitstops.duplicated().sum() # no duplicates found

0

## Summary statistics

In [10]:
df_pitstops.describe()

,race_id,gp_year,gp_round,driver_id,stop_number,lap_number,pit_duration_ms,pit_duration_s
count,1286.000000,1286.000000,1286.000000,1286.000000,1286.000000,1286.000000,1.286000e+03,1286.000000
mean,979.593313,2017.066096,10.631415,645.486781,1.721617,23.739502,5.903852e+04,59.038520
std,28.022674,1.309463,5.919124,321.400165,0.987998,14.687026,2.190186e+05,219.018611
min,926.000000,2015.000000,1.000000,9.000000,1.000000,1.000000,1.495100e+04,14.951000
25%,957.000000,2016.000000,5.000000,807.000000,1.000000,12.000000,2.235475e+04,22.354750
50%,977.500000,2017.000000,11.000000,815.000000,1.000000,23.000000,2.386300e+04,23.863000
75%,1003.000000,2018.000000,15.000000,832.000000,2.000000,34.000000,2.664750e+04,26.647500
max,1030.000000,2019.000000,21.000000,847.000000,6.000000,72.000000,2.011147e+06,2011.147000


From this we can roughly tell that, 
- Most pit stops occur early or mid-race. Median lap no. is 23, with majority between 12 and 34.
- Mean pit duration is 59.04 s. This is inflated by extreme outliers.
- Median pit duration is 23.86 s, far more realistic.
- Most pit stops fall between 22.35 s (25th percentile) and 26.64 s (75th percentile)

- Max duration of 2011.147 s, or nearly 34 minutes, is clearly abnormal. Likely a retired car or incorrectly logged time. Should be flagged or removed.
- Max pit stops number of 6 is unusual, as most cars pit only 1-3 times. Could signal a chaotic race, multiple penalties, or heavy tyre degradation race.

## Drop, or flag outliers?

- This dataset will be heavily used with Fast-F1, understanding safety cars and VSC periods.
- Doing so, I'm inclined not to drop, but actually flag, extreme pit durations of over 90s or abnormal strategies of more than 3 stops.
- This will help analyse context like safety cars, weather, chaotic race conditions (e.g. Germany 2019).

In [37]:
df_pitstops['long_stop_flag'] = df_pitstops['pit_duration_s'] > 90 # any pit stops that last longer than 90 seconds are flagged as long stops
df_pitstops['multi_stops_flag'] = df_pitstops['stop_number'] > 3 # any pit stops that are more than 3 are flagged as multi stops
df_pitstops['chaotic_race_flag'] = df_pitstops['long_stop_flag'] | df_pitstops['multi_stops_flag'] # any pit stops that are either multi stops or long stops are flagged as chaotic

In [38]:
df_pitstops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1286 entries, 0 to 1285
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   race_id            1286 non-null   int64  
 1   gp_year            1286 non-null   int64  
 2   gp_name            1286 non-null   object 
 3   gp_round           1286 non-null   int64  
 4   driver_id          1286 non-null   int64  
 5   driver_name        1286 non-null   object 
 6   constructor        1286 non-null   object 
 7   constructor_ref    1286 non-null   object 
 8   is_williams        1286 non-null   bool   
 9   stop_number        1286 non-null   int64  
 10  lap_number         1286 non-null   int64  
 11  time_of_stop       1286 non-null   object 
 12  pit_duration       1286 non-null   object 
 13  pit_duration_ms    1286 non-null   int64  
 14  pit_duration_s     1286 non-null   float64
 15  long_stop_flag     1286 non-null   bool   
 16  multi_stops_flag   1286 

In [39]:
# Access observations with long pit stops
long_stops = df_pitstops[df_pitstops['long_stop_flag']]
print(long_stops.head())

     race_id  gp_year                gp_name  gp_round  driver_id  \
151      948     2016  Australian Grand Prix         1        807   
152      948     2016  Australian Grand Prix         1        815   
153      948     2016  Australian Grand Prix         1        154   
157      948     2016  Australian Grand Prix         1        835   
158      948     2016  Australian Grand Prix         1        825   

         driver_name   constructor constructor_ref  is_williams  stop_number  \
151  Nico Hülkenberg   Force India     force_india        False            2   
152     Sergio Pérez   Force India     force_india        False            2   
153  Romain Grosjean  Haas F1 Team            haas        False            1   
157    Jolyon Palmer       Renault         renault        False            2   
158  Kevin Magnussen       Renault         renault        False            3   

     lap_number time_of_stop pit_duration  pit_duration_ms  pit_duration_s  \
151          18     16:37:

In [40]:
# Access observations with multi stops
multi_stops = df_pitstops[df_pitstops['multi_stops_flag']]
print(multi_stops.head())

    race_id  gp_year               gp_name  gp_round  driver_id  \
71      936     2015  Hungarian Grand Prix        10        815   
72      936     2015  Hungarian Grand Prix        10        815   
79      936     2015  Hungarian Grand Prix        10         13   
80      936     2015  Hungarian Grand Prix        10        822   
81      936     2015  Hungarian Grand Prix        10        822   

        driver_name  constructor constructor_ref  is_williams  stop_number  \
71     Sergio Pérez  Force India     force_india        False            4   
72     Sergio Pérez  Force India     force_india        False            5   
79     Felipe Massa     Williams        williams         True            4   
80  Valtteri Bottas     Williams        williams         True            4   
81  Valtteri Bottas     Williams        williams         True            5   

    lap_number time_of_stop pit_duration  pit_duration_ms  pit_duration_s  \
71          44     15:14:51       16.958           

In [45]:
# Access observations with long pit stops AND multi stops AND choatic race flag
chaotic_stops = df_pitstops[df_pitstops['long_stop_flag'] & df_pitstops['multi_stops_flag'] & df_pitstops['chaotic_race_flag']]
print(chaotic_stops.head())

     race_id  gp_year                gp_name  gp_round  driver_id  \
617      976     2017  Azerbaijan Grand Prix         8        839   
625      976     2017  Azerbaijan Grand Prix         8        154   

         driver_name   constructor constructor_ref  is_williams  stop_number  \
617     Esteban Ocon   Force India     force_india        False            4   
625  Romain Grosjean  Haas F1 Team            haas        False            4   

     lap_number time_of_stop pit_duration  pit_duration_ms  pit_duration_s  \
617          22     17:53:52    21:53.873          1313873        1313.873   
625          22     17:53:42    21:53.665          1313665        1313.665   

     long_stop_flag  multi_stops_flag  chaotic_race_flag  
617            True              True               True  
625            True              True               True  


In [47]:
df_pitstops.to_csv('/Users/frankdong/Documents/Analytics Local/williams-racing-strategies/processed_data/constructor_pit_stops_validated.csv', index=False)

## Validation conclusion
- No null values found
- Column data types are correct
- Long or multiple pit stops are flagged, as well as chaotic races, dependent on the two new columns.
- Proceed with feature engineering using the provided CSV data in 'constructor_pit_stops_validated.csv'